In [1]:
from pprint import pprint
import pandas as pd
from gensim import corpora
import gensim
import logging
import re
import multiprocessing
import collections
from tqdm import tqdm
from collections import defaultdict
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [80]:
fname = 'joblistings.merged.parsed.unique.grpbyyear.2010-2015.01.tsv'
cores = multiprocessing.cpu_count()
print cores

8


In [196]:
def get_corpus(fname):
    with open(fname) as f:
        lines = iter(f)
        lines.next()
        for line in lines:
            yield line
            
class CorpusFriendly(object):
    def __init__(self,fname, topK=None):
        self.topK = topK
        self.fname = fname
        
    def __iter__(self):
        k = 1
        corpus_memory_friendly = get_corpus(self.fname)
        for data in corpus_memory_friendly:
            data = gensim.utils.to_unicode(data).split(',')
            #print data
            words = data[2].split()
            label = data[0]
            yield gensim.models.doc2vec.TaggedDocument(words, [label])
            if self.topK:
                if k >= self.topK:
                    break
                k += 1
        

In [149]:
# print model.similarity('computer', 'computer')
# print model.similarity('programming', 'programmer')
# print model.similarity('programming', 'engineering') 

In [226]:
model.most_similar(positive=['engineering','programming'], topn=10)

[(u'cad', 0.35921186208724976),
 (u'hardware', 0.336124062538147),
 (u'editing', 0.33407896757125854),
 (u'optics', 0.32418403029441833),
 (u'graphic', 0.32240891456604004),
 (u'auditing', 0.3200691044330597),
 (u'unix', 0.31896811723709106),
 (u'engineer', 0.31864529848098755),
 (u'design', 0.29961076378822327),
 (u'troubleshooting', 0.2965896725654602)]

In [227]:
model2.most_similar(positive=['engineering','programming'], topn=10)

[(u'cad', 0.3522326946258545),
 (u'hardware', 0.330003023147583),
 (u'auditing', 0.31466442346572876),
 (u'design', 0.30814027786254883),
 (u'troubleshooting', 0.3070387840270996),
 (u'instrumentation', 0.3017618656158447),
 (u'biology', 0.29656803607940674),
 (u'autocad', 0.29571235179901123),
 (u'electrical', 0.2952849268913269),
 (u'jvm', 0.2948858141899109)]

In [228]:
model_full_major.most_similar(positive=['engineering','programming'], topn=10)

[(u'hardware', 0.4103742241859436),
 (u'architecture', 0.33160144090652466),
 (u'embedded', 0.3228675127029419),
 (u'development', 0.3150022625923157),
 (u'design', 0.30437466502189636),
 (u'integration', 0.3014294505119324),
 (u'modeling', 0.2951388955116272),
 (u'accounting', 0.29434525966644287),
 (u'programing', 0.2870217561721802),
 (u'spreadsheet', 0.2822428345680237)]

## Train on 5000 documents
**model 1:** paragraph_id (tag) = onet major group code, which is the first 2 digits  
**model 2:** paragraph_id (tag) = whole onet code, which is something like 13-1141.00

In [186]:
model = gensim.models.Word2Vec.load("gensim_doc2vec_topK5000")  # open the model
model2 = gensim.models.Word2Vec.load("gensim_doc2vec_topK5000_onetcode")  # open the model

2016-12-01 17:30:55,731 : INFO : loading Word2Vec object from gensim_doc2vec_topK5000
2016-12-01 17:30:55,971 : INFO : loading docvecs recursively from gensim_doc2vec_topK5000.docvecs.* with mmap=None
2016-12-01 17:30:55,971 : INFO : loading syn0 from gensim_doc2vec_topK5000.syn0.npy with mmap=None
2016-12-01 17:30:56,090 : INFO : loading syn1 from gensim_doc2vec_topK5000.syn1.npy with mmap=None
2016-12-01 17:30:56,282 : INFO : setting ignored attribute syn0norm to None
2016-12-01 17:30:56,287 : INFO : setting ignored attribute cum_table to None
2016-12-01 17:30:56,294 : INFO : loading Word2Vec object from gensim_doc2vec_topK5000_onetcode
2016-12-01 17:30:56,537 : INFO : loading docvecs recursively from gensim_doc2vec_topK5000_onetcode.docvecs.* with mmap=None
2016-12-01 17:30:56,537 : INFO : loading syn0 from gensim_doc2vec_topK5000_onetcode.syn0.npy with mmap=None
2016-12-01 17:30:56,651 : INFO : loading syn1 from gensim_doc2vec_topK5000_onetcode.syn1.npy with mmap=None
2016-12-01 17

### Model 1 Evaluation

In [202]:
corpus_test = CorpusFriendly(fname="va_test_clean.csv")

In [208]:
ranks = []
for des in tqdm(corpus_test):
    inferred_vector = model.infer_vector(des.words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(des.tags[0])
#     print des.tags[0]
    ranks.append(rank)
#     print sims
#     break

72000it [25:23, 47.25it/s]


In [209]:
rank_count = collections.Counter(ranks)
print rank_count

Counter({0: 43016, 1: 9997, 2: 5014, 3: 2823, 4: 1846, 5: 1437, 6: 1055, 7: 898, 8: 789, 9: 665, 10: 588, 11: 571, 12: 448, 13: 411, 14: 383, 15: 336, 16: 299, 17: 250, 18: 227, 19: 209, 20: 208, 21: 179, 22: 159, 23: 116, 24: 76})


In [210]:
# Top N Accuracy
print "Top-1 Accuracy", sum(rank_count.values()[:1]) / float(sum(collections.Counter(ranks).values()))
print "Top-2 Accuracy", sum(rank_count.values()[:2]) / float(sum(collections.Counter(ranks).values()))
print "Top-3 Accuracy", sum(rank_count.values()[:3]) / float(sum(collections.Counter(ranks).values()))
print "Top-4 Accuracy", sum(rank_count.values()[:4]) / float(sum(collections.Counter(ranks).values()))
print "Top-5 Accuracy", sum(rank_count.values()[:5]) / float(sum(collections.Counter(ranks).values()))

Top-1 Accuracy 0.597444444444
Top-2 Accuracy 0.736291666667
Top-3 Accuracy 0.805930555556
Top-4 Accuracy 0.845138888889
Top-5 Accuracy 0.870777777778


### Model 2 Evaluation

In [ ]:
corpus_test = CorpusFriendly(fname="va_test_clean.csv")

In [195]:
ranks2 = []
for des in tqdm(corpus_test):
    inferred_vector = model2.infer_vector(des.words)
    sims = model2.docvecs.most_similar([inferred_vector], topn=len(model2.docvecs))
    rank = [docid[:2] for docid, sim in sims].index(des.tags[0])
    #print des.tags[0]
    ranks2.append(rank)
    #print ranks

72000it [29:06, 33.28it/s]


In [197]:
rank2_count = collections.Counter(ranks2)
print rank2_count

Counter({0: 41606, 1: 7850, 2: 4240, 3: 2565, 4: 1670, 5: 1322, 6: 1037, 7: 884, 8: 693, 9: 618, 10: 544, 11: 518, 12: 465, 13: 420, 14: 379, 15: 328, 16: 316, 17: 289, 18: 267, 19: 237, 20: 233, 21: 232, 22: 204, 24: 196, 23: 185, 26: 160, 27: 149, 25: 144, 28: 137, 30: 114, 32: 114, 31: 106, 29: 105, 33: 91, 36: 90, 40: 87, 39: 83, 37: 82, 34: 80, 45: 77, 35: 76, 41: 76, 46: 73, 38: 72, 42: 66, 48: 66, 44: 59, 47: 59, 50: 59, 43: 57, 54: 57, 49: 49, 58: 49, 53: 48, 51: 46, 57: 41, 60: 40, 59: 39, 66: 39, 52: 38, 67: 38, 61: 37, 62: 37, 65: 37, 55: 35, 63: 35, 64: 35, 68: 35, 56: 33, 76: 33, 78: 31, 70: 30, 73: 30, 77: 29, 71: 28, 75: 27, 80: 27, 82: 27, 81: 26, 72: 25, 74: 25, 101: 25, 88: 24, 85: 23, 89: 22, 103: 22, 83: 21, 94: 21, 112: 21, 69: 20, 87: 20, 92: 20, 79: 19, 93: 19, 86: 18, 90: 18, 91: 18, 109: 18, 96: 17, 99: 17, 104: 17, 95: 16, 105: 16, 97: 15, 100: 15, 116: 15, 120: 15, 130: 15, 84: 14, 118: 14, 98: 13, 102: 13, 111: 13, 117: 13, 107: 12, 108: 12, 110: 12, 121: 12

In [198]:
# Top N Accuracy
print "Top-1 Accuracy", sum(rank2_count.values()[:1]) / float(sum(collections.Counter(ranks2).values()))
print "Top-2 Accuracy", sum(rank2_count.values()[:2]) / float(sum(collections.Counter(ranks2).values()))
print "Top-3 Accuracy", sum(rank2_count.values()[:3]) / float(sum(collections.Counter(ranks2).values()))
print "Top-4 Accuracy", sum(rank2_count.values()[:4]) / float(sum(collections.Counter(ranks2).values()))
print "Top-5 Accuracy", sum(rank2_count.values()[:5]) / float(sum(collections.Counter(ranks2).values()))

Top-1 Accuracy 0.577861111111
Top-2 Accuracy 0.686888888889
Top-3 Accuracy 0.745777777778
Top-4 Accuracy 0.781402777778
Top-5 Accuracy 0.804597222222


## Training on full dataset 288000 documents

In [212]:
corpus_test = CorpusFriendly(fname="va_test_clean.csv")

In [211]:
model_full_major = gensim.models.Word2Vec.load("gensim_doc2vec_major")  # open the model

2016-12-01 19:15:32,878 : INFO : loading Word2Vec object from gensim_doc2vec_major
2016-12-01 19:15:34,974 : INFO : loading docvecs recursively from gensim_doc2vec_major.docvecs.* with mmap=None
2016-12-01 19:15:34,975 : INFO : loading syn0 from gensim_doc2vec_major.syn0.npy with mmap=None
2016-12-01 19:15:35,547 : INFO : loading syn1 from gensim_doc2vec_major.syn1.npy with mmap=None
2016-12-01 19:15:36,232 : INFO : setting ignored attribute syn0norm to None
2016-12-01 19:15:36,233 : INFO : setting ignored attribute cum_table to None


In [213]:
ranks_full = []
for des in tqdm(corpus_test):
    inferred_vector = model.infer_vector(des.words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(des.tags[0])
#     print des.tags[0]
    ranks_full.append(rank)
#     print sims
#     break

72000it [14:25, 83.21it/s]


In [214]:
rank_full_count = collections.Counter(ranks_full)
print rank_full_count

Counter({0: 42941, 1: 9981, 2: 5004, 3: 2942, 4: 1855, 5: 1349, 6: 1106, 7: 877, 8: 724, 9: 687, 10: 616, 11: 512, 12: 497, 13: 409, 14: 390, 15: 344, 16: 296, 18: 269, 17: 260, 19: 245, 20: 205, 21: 165, 22: 126, 23: 108, 24: 92})


In [215]:
# Top N Accuracy
print "Top-1 Accuracy", sum(rank_full_count.values()[:1]) / float(sum(collections.Counter(ranks_full).values()))
print "Top-2 Accuracy", sum(rank_full_count.values()[:2]) / float(sum(collections.Counter(ranks_full).values()))
print "Top-3 Accuracy", sum(rank_full_count.values()[:3]) / float(sum(collections.Counter(ranks_full).values()))
print "Top-4 Accuracy", sum(rank_full_count.values()[:4]) / float(sum(collections.Counter(ranks_full).values()))
print "Top-5 Accuracy", sum(rank_full_count.values()[:5]) / float(sum(collections.Counter(ranks_full).values()))

Top-1 Accuracy 0.596402777778
Top-2 Accuracy 0.735027777778
Top-3 Accuracy 0.804527777778
Top-4 Accuracy 0.845388888889
Top-5 Accuracy 0.871152777778


# Snippet examples

In [263]:
onetdict ={'11': 'Management Occupations',
           '13': 'Business and Financial Operations Occupations',
           '15': 'Computer and Mathematical Occupations',
           '17': 'Architecture and Engineering Occupations',
           '19': 'Life, Physical, and Social Science Occupations',
           '21': 'Community and Social Service Occupations',
           '23': 'Legal Occupations',
           '25': 'Education, Training, and Library Occupations',
           '27': 'Arts, Design, Entertainment, Sports, and Media Occupations',
           '29': 'Healthcare Practitioners and Technical Occupations',
           '31': 'Healthcare Support Occupations',
           '33': 'Protective Service Occupations',
           '35': 'Food Preparation and Serving Related Occupations',
           '37': 'Building and Grounds Cleaning and Maintenance',
           '39': 'Personal Care and Service Occupations',
           '41': 'Sales and Related Occupations', 
           '43': 'Office and Administrative Support Occupations',
           '45': 'Farming, Fishing, and Forestry Occupations',
           '47': 'Construction and Extraction Occupations', 
           '49': 'Installation, Maintenance, and Repair Occupations', 
           '51': 'Production Occupations', 
           '53': 'Transportation and Material Moving Occupations', 
           '55': 'Military Specific Occupations',
           '99': 'Unknown'}

In [164]:
test_list = list(corpus_test.__iter__())

In [ ]:
# test id = 45, 3123, 4646

In [260]:
doc_id = 3123
print('Document ({}) Major Group ({}) {}: \n«{}»\n'.format(doc_id, test_list[doc_id].tags[0], onetdict[test_list[doc_id].tags[0]],' '.join(test_list[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
inferred_vector = model.infer_vector(test_list[doc_id].words)
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
print sims[0], onetdict[sims[0][0]]
print sims[-1], onetdict[sims[-1][0]]


Document (3123) Major Group (13) Business and Financial Operations Occupations: 
«barista takeda pharmaceuticals international company personal financial advisors span strong about aramark strong span br span aramark nyse armk is in the customer service business across food facilities and uniforms wherever people work learn recover and play united by passion to serve our more than employees deliver experiences that enrich and nourish the lives of millions of people in countries around the world every day aramark is recognized among the most admired companies by fortune and the worlds most ethical companies by the ethisphere institute learn more at www aramark com or connect with us on facebook and twitter span br span strong essential tasks and strong span ul li span maintain positive attitude and smile and be polite with customers throughout the transaction span li li span greet customers as they approach the cash register span li li span take the customers order and prepare the order

In [261]:
doc_id = 3123
print('Document ({}) Major Group ({}) {}: \n«{}»\n'.format(doc_id, test_list[doc_id].tags[0][:2], onetdict[test_list[doc_id].tags[0][:2]],' '.join(test_list[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model2)
inferred_vector = model2.infer_vector(test_list[doc_id].words)
sims = model2.docvecs.most_similar([inferred_vector], topn=len(model2.docvecs))
print sims[0], onetdict[sims[0][0][:2]]
print sims[-1], onetdict[sims[-1][0][:2]]


Document (3123) Major Group (13) Business and Financial Operations Occupations: 
«barista takeda pharmaceuticals international company personal financial advisors span strong about aramark strong span br span aramark nyse armk is in the customer service business across food facilities and uniforms wherever people work learn recover and play united by passion to serve our more than employees deliver experiences that enrich and nourish the lives of millions of people in countries around the world every day aramark is recognized among the most admired companies by fortune and the worlds most ethical companies by the ethisphere institute learn more at www aramark com or connect with us on facebook and twitter span br span strong essential tasks and strong span ul li span maintain positive attitude and smile and be polite with customers throughout the transaction span li li span greet customers as they approach the cash register span li li span take the customers order and prepare the order

In [277]:
doc_id = 3123
print('Document ({}) Major Group ({}) {}: \n«{}»\n'.format(doc_id, test_list[doc_id].tags[0], onetdict[test_list[doc_id].tags[0]],' '.join(test_list[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model_full_major)
inferred_vector = model_full_major.infer_vector(test_list[doc_id].words)
sims = model_full_major.docvecs.most_similar([inferred_vector], topn=len(model_full_major.docvecs))
print sims[0], onetdict[sims[0][0]]
print sims[-1], onetdict[sims[-1][0]]

Document (3123) Major Group (13) Business and Financial Operations Occupations: 
«barista takeda pharmaceuticals international company personal financial advisors span strong about aramark strong span br span aramark nyse armk is in the customer service business across food facilities and uniforms wherever people work learn recover and play united by passion to serve our more than employees deliver experiences that enrich and nourish the lives of millions of people in countries around the world every day aramark is recognized among the most admired companies by fortune and the worlds most ethical companies by the ethisphere institute learn more at www aramark com or connect with us on facebook and twitter span br span strong essential tasks and strong span ul li span maintain positive attitude and smile and be polite with customers throughout the transaction span li li span greet customers as they approach the cash register span li li span take the customers order and prepare the order